In [35]:
import sdv
import pandas as pd
from sdv.single_table.ctgan import CTGAN
from sdv.metadata import SingleTableMetadata
import logging
import random
import string

In [26]:
client = "national_scleroderma_foundation"
path = "/home/RMittal@ccsfundraising.com/ccs_pred_mod"
filename =  "%s_constituent_ccsdb.csv" %(client)
file = "%s/%s" %(path, filename)
df_cd = pd.read_csv(file, encoding="ISO-8859-1")
df_cd = df_cd.dropna(how="all", axis=1)
df_cd = df_cd.rename(columns={"solicit_codes": "solicit_codes_binary"})
df_cd["solicit_codes_binary"] = df_cd["solicit_codes_binary"].astype("bool")
df_cd = df_cd.drop(columns=["Unnamed: 0"])
df_cd["spouse_age"] = pd.to_datetime(df_cd["spouse_age"])

/tmp/ipykernel_28266/837933114.py:5: DtypeWarning: Columns (6,19,22,24,26,27,29,31,32,35,39,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cd = pd.read_csv(file, encoding="ISO-8859-1")


In [34]:
len(df_cd)

248443

In [27]:
df_cd[df_cd.select_dtypes(include='object').columns] = df_cd.select_dtypes(include='object')\
                                                       .apply(lambda x: x.str.replace("scleroderma", "Company", case=False))

In [41]:
columns_nans ={key: df_cd[key].isna().sum() for key in df_cd.columns}


In [ ]:
def create_array_with_nans(numbers, nans):
    # Combine the numbers and NaNs
    combined_list = numbers + [np.nan] * nans
    
    # Shuffle the combined list randomly
    np.random.shuffle(combined_list)
    
    # Convert the list to a numpy array
    result_array = np.array(combined_list)
    
    return result_array

# Example usage
N1 = 5
N2 = 3
numbers_or_strings = [1, 2, "Alice", "Bob", 3.14]
result = create_array_with_nans(numbers_or_strings, N2)

In [43]:
# Generate a list of 100 random strings
def generate_random_strings(num_strings, string_length):
    characters = string.ascii_letters + string.digits
    return [''.join(random.choice(characters) for _ in range(string_length)) for _ in range(num_strings)]

In [55]:
# Addresses

columns_address = ["address_1", "business_address", "seasonal_address"]

for col in columns_address:

    N = len(df_cd) - columns_nans[col]

    # Generate a list of N random integers
    syn_house_numbers = [random.randint(1, N) for _ in range(N)]

    # Example: Generate a list of N random strings, each with a length of 10 characters
    syn_street_names = generate_random_strings(num_strings=N, string_length=10)

    # List of street suffixes
    suffixes = ["Ave", "Str", "Rd"]
    syn_suffix = [random.choice(suffixes) for _ in range(N)]

    globals()["%s_syn" %col] = [f"{num} {name} {suffix}" for num, name, suffix \
                        in zip(syn_house_numbers, syn_street_names, syn_suffix)]

In [ ]:
# Address_2

df_cd["address_2"] = df_cd["address_2"].str.replace("apt", "Ste", case=False)
df_cd["address_2"] = df_cd["address_2"].str.replace("ste", "Apt", case=False)

In [61]:
# Emails

columns_email = ["personal_email", "spouse_email"]

for col in columns_email:

    N = len(df_cd) - columns_nans[col]

    # Example: Generate a list of N random strings, each with a length of 5 characters
    syn_first_string = generate_random_strings(num_strings=N, string_length=5)

    # Example: Generate a list of N random strings, each with a length of 5 characters
    syn_second_string = generate_random_strings(num_strings=N, string_length=5)

    # List of street suffixes
    suffixes = ["@gmail.com", "@yahoo.com", "@verizon.net", "@outlook.com"]
    syn_suffix = [random.choice(suffixes) for _ in range(N)]

    globals()["%s_syn" %col] = [f"{first}.{second}{suffix}" for first, second, suffix \
                        in zip(syn_first_string, syn_second_string, syn_suffix)]

In [ ]:
# All Other Fields

columns_to_synthesize_string = ["first_name", "middle_name", "last_name", "spouse_name", "spouse_business_name"]
for col in columns_to_synthesize_string:

    N = len(df_cd) - columns_nans[col]
    globals()["%s_syn" %col] = generate_random_strings(num_strings=N, string_length=10)

columns_to_synthesize_integer = ["constituent_id", "spouse_id", "home_phone", "cell_phone", "business_phone"]
for col in columns_to_synthesize_integer:

    N = len(df_cd) - columns_nans[col]
    if "phone" in col:
        globals()["%s_syn" %col] = [random.randint(1000000000, 9999999999) for _ in range(N)]
    else:
        globals()["%s_syn" %col] = [random.randint(1, N) for _ in range(N)]

In [ ]:
for col in columns_to_synthesize:
    N = len(df_cd) - columns_nans[col]
    globals()["%s_syn" %col] = generate_random_strings(num_strings=N, string_length=10)

In [89]:
#nsf_dtypes = pd.read_csv("nsf_dtypes.txt", sep='\s+,')
# for idx, row in nsf_dtypes.iterrows():
#     field = row["fields"]
#     dtype = row["dtypes"]
    
#     if dtype == "str":
#         df_cd[field] = df_cd[field].astype(str)
#     elif dtype == "datetime":
#         df_cd[field] = pd.to_datetime(df_cd[field], errors='coerce')
#     elif dtype == "float64":
#         df_cd[field] = df_cd[field].astype(float, errors='ignore')
#     elif dtype == "int":
#         df_cd[field] = df_cd[field].astype(int, errors='ignore')

In [ ]:
#df_subset = df_cd.iloc[:, 1:5]
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df_cd)
python_dict = metadata.to_dict()

In [91]:
python_dict['columns']['middle_name'] = {
    "sdtype": "middle_name",
    "pii": "True"
}
python_dict['columns']['address_1'] = {
    "sdtype": "text",
    "pii": "True"
}
# Update metadata
metadata = SingleTableMetadata.load_from_dict(python_dict)

In [93]:
metadata.validate_data(data=df_subset)
synthesizer = CTGAN(
    metadata,
    verbose=True
)
synthesizer.fit(df_subset)